In [172]:
import requests
import networkx as nx
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

In [173]:
def load_txt_file(file_name, columns, delimiter):
    data = pd.read_csv(file_name , delimiter=delimiter, names= columns)
    return data

In [174]:
columns = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport'
           , 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']
airports_data = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude'
                 , 'Altitude', 'Timezone', 'DST', 'Tz Database time', 'Type', 'Source']
data = load_txt_file('routes.dat.txt', columns, delimiter=',')
lng_lot_data = load_txt_file('airports.dat.txt', airports_data, delimiter=',')
data  = data.dropna()

In [175]:
iata_hash = {}
for index, row in lng_lot_data.iterrows():
    iata_hash[row["IATA"]] = {'Longitude':row['Longitude'], 'Latitude':row['Latitude']} 
def g_lg(key):
    if key in iata_hash:
        hash_iata = iata_hash.get(key)
        return hash_iata['Latitude'], hash_iata['Longitude']
    return (0,0)

In [176]:
G = nx.DiGraph()
graph_data = [i for i in zip(data['Source airport'].tolist(), data['Destination airport'].tolist())]
graph_data = [( g_data[0],g_data[1], geopy.distance.vincenty(g_lg(g_data[0]), g_lg(g_data[1])).km ) for g_data in graph_data]
G.add_weighted_edges_from(graph_data)

/home/muhammadumar/anaconda3/envs/ml_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  This is separate from the ipykernel package so we can avoid doing imports until


In [183]:
weight = nx.dijkstra_path_length(G, 'LHR', 'CLT')
paths = nx.dijkstra_path(G, 'LHR', 'CLT')
print("Path = ", paths)
print("Distance = ", weight, "KM")

Path =  ['LHR', 'EWR', 'CLT']
Distance =  6427.121672839762 KM
